<a href="https://colab.research.google.com/github/dyou-wbg/meta_processing/blob/main/04_26_Annotation_2KPair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
from google.colab import drive
import csv
import os
import re
import pandas as pd
import numpy as np

directory = '/content/drive/MyDrive/finalMetaFile'
df = pd.read_csv(os.path.join(directory, 'combined_metadata_final12_features.csv'))
df=df[['sourcename', 'title']]


df['sourcename'] = df['sourcename'].apply(lambda x: x.replace('metadata_', ''))
df['sourcename'] = df['sourcename'].apply(lambda x: x.replace('_unifiedColName.csv', ''))
df['sourcename'].replace(['url1_ie_table_complete', 'scopus_fixed_encoding','SSRN'], ['3ie', 'scopus','ssrn'], inplace=True)


Mounted at /content/drive


<ipython-input-1-54ccb68a7028>:18: DtypeWarning: Columns (1,2,3,4,5,7,9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(directory, 'combined_metadata_final12_features.csv'))


In [5]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize

df['title'] = df['title'].str.lower()

def remove_non_english(df):
  def is_english(text):
    words = word_tokenize(text)
    english_word_count = sum(len(word) >= 2 for word in words)
    return english_word_count / len(words) >= 0.9

  df_filtered = df[df['title'].apply(is_english)]
  return df_filtered

df['title'] = df['title'].astype(str)
df = df[~df['title'].str.contains('و|д|я|š|ü|à|é|ś|ä|ú|의|θ|λ|ό|τ|η|τ|α|の|と', regex=True)]
df = remove_non_english(df.copy())
def count_words(title):
  words = title.split()
  return len(words)

df['word_count'] = df['title'].apply(count_words)
df.to_csv(os.path.join(directory, 'combined_metadata_final12_features_MakePair.csv'), index=False)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:

# Filter the DataFrame by sourcename.
df_3ie = df[df['sourcename'] == '3ie']
df_econlit = df[df['sourcename'] == 'econlit']
df_ssrn = df[df['sourcename'] == 'ssrn']
df_elsevier = df[df['sourcename'] == 'elsevier']
df_repec = df[df['sourcename'] == 'repec']
df_scopus = df[df['sourcename'] == 'scopus']
df_jstor = df[df['sourcename'] == 'jstor']
df_semantic_scholar = df[df['sourcename'] == 'semantic_scholar']
df_repec = df_repec[df_repec['word_count'] < 1000]

In [7]:
min=5
max=20

df_3ie_min_max = df_3ie[(df_3ie['word_count'] >= min) & (df_3ie['word_count'] <= max)]
df_econlit_min_max = df_econlit[(df_econlit['word_count'] >= min) & (df_econlit['word_count'] <= max)]
df_ssrn_min_max = df_ssrn[(df_ssrn['word_count'] >= min) & (df_ssrn['word_count'] <= max)]
df_elsevier_min_max = df_elsevier[(df_elsevier['word_count'] >= min) & (df_elsevier['word_count'] <= max)]
df_repec_min_max = df_repec[(df_repec['word_count'] >= min) & (df_repec['word_count'] <= max)]
df_scopus_min_max = df_scopus[(df_scopus['word_count'] >= min) & (df_scopus['word_count'] <= max)]
df_jstor_min_max = df_jstor[(df_jstor['word_count'] >= min) & (df_jstor['word_count'] <= max)]
df_semantic_scholar_min_max = df_semantic_scholar[(df_semantic_scholar['word_count'] >= min) & (df_semantic_scholar['word_count'] <= max)]

def sample_dataframe(df, sample_size):
  if len(df) > sample_size:
    return df.sample(n=sample_size)
  else:
    return df

import math

title_sample_size=round(math.sqrt(2000)/8)

df_3ie_min_max = sample_dataframe(df_3ie_min_max, title_sample_size)
df_econlit_min_max = sample_dataframe(df_econlit_min_max, title_sample_size)
df_ssrn_min_max = sample_dataframe(df_ssrn_min_max, title_sample_size)
df_elsevier_min_max = sample_dataframe(df_elsevier_min_max, title_sample_size)
df_repec_min_max = sample_dataframe(df_repec_min_max, title_sample_size)
df_scopus_min_max = sample_dataframe(df_scopus_min_max, title_sample_size)
df_jstor_min_max = sample_dataframe(df_jstor_min_max, title_sample_size)
df_semantic_scholar_min_max = sample_dataframe(df_semantic_scholar_min_max, title_sample_size)

df_make_pair = pd.concat([df_3ie_min_max, df_econlit_min_max, df_ssrn_min_max, df_elsevier_min_max, df_repec_min_max, df_scopus_min_max, df_jstor_min_max, df_semantic_scholar_min_max])

#df_make_pair.to_csv(os.path.join(directory, 'combined_metadata_final12_features_MakePair_annotation.csv'), index=False)

In [9]:
df_make_pair.head()

,sourcename,title,word_count
10217341,3ie,neither a zinc supplement nor phytate-reduced ...,18
10220313,3ie,the effects of social norms on motivation crow...,14
10211800,3ie,evaluating the environmental impact of payment...,18
10222072,3ie,saving lives with pre-arranged disaster aid: e...,9
10222290,3ie,conditional cash transfers and child labor,6


In [8]:
import matplotlib.pyplot as plt

df_make_pair['sourcename'].value_counts().plot(kind='bar')
plt.xlabel('Source Name')
plt.ylabel('Count')
plt.title('Distribution of Source Names')
plt.show()

In [10]:
# prep to make a pair

from string import printable
from collections import Counter

printable_dict = {c:i for i,c in enumerate(printable)}

def vectorize(title):
    cnt = Counter(title)
    array = [0 for _ in printable]+[0]
    for c,n in cnt.items():
        if c in printable_dict:
            array[printable_dict[c]]=n
        else:
            array[-1]+=n
    return np.array(array)

vects = []
for title in df_make_pair.title:
    vects.append(vectorize(title))

In [11]:
threshold = 1000 # force very high to return all possible pair
min_length = 5 # was 10 but change to 5 to keep all sampled titles


with open('/content/drive/MyDrive/finalMetaFile/counterVect_L1_2kPair_annotation.csv', 'w', newline='') as file:
  writer = csv.writer(file)
  writer.writerow(["Title 1", "Title 2", "Similarity Score"])
# Iterate over title pairs
  for i, v1 in enumerate(vects):
    if sum(v1[:-1]) > min_length: # Only above a minimum number of printable characters
        for j, v2 in enumerate(vects):
            if sum(v2[:-1]) > min_length:
                if i!=j:
                    l1 = np.linalg.norm((v1[:-1] - v2[:-1]), ord=1)
                    if l1 < threshold:
                        t1 = df_make_pair.iloc[i].title
                        t2 = df_make_pair.iloc[j].title
                        writer.writerow([t1, t2, l1])
                        print(f"{t1} _&_ {t2} (L1: {l1:.0f})")

neither a zinc supplement nor phytate-reduced maize nor their combination enhance growth of 6- to 12-month-old guatemalan infants _&_ the effects of social norms on motivation crowding: experimental evidence from the tibetan plateau (L1: 45)
neither a zinc supplement nor phytate-reduced maize nor their combination enhance growth of 6- to 12-month-old guatemalan infants _&_ evaluating the environmental impact of payments for ecosystem services in coatepec (mexico) using remote sensing and on-site interviews (L1: 54)
neither a zinc supplement nor phytate-reduced maize nor their combination enhance growth of 6- to 12-month-old guatemalan infants _&_ saving lives with pre-arranged disaster aid: evidence from mexico (L1: 80)
neither a zinc supplement nor phytate-reduced maize nor their combination enhance growth of 6- to 12-month-old guatemalan infants _&_ conditional cash transfers and child labor (L1: 89)
neither a zinc supplement nor phytate-reduced maize nor their combination enhance gr

In [12]:
df_annotation = pd.read_csv(os.path.join(directory, 'counterVect_L1_2kPair_annotation.csv'))
df_annotation['annotation_label']=0
df_annotation.head()

,Title 1,Title 2,Similarity Score,annotation_label
0,neither a zinc supplement nor phytate-reduced ...,the effects of social norms on motivation crow...,45.0,0
1,neither a zinc supplement nor phytate-reduced ...,evaluating the environmental impact of payment...,54.0,0
2,neither a zinc supplement nor phytate-reduced ...,saving lives with pre-arranged disaster aid: e...,80.0,0
3,neither a zinc supplement nor phytate-reduced ...,conditional cash transfers and child labor,89.0,0
4,neither a zinc supplement nor phytate-reduced ...,do households gain from community-based natura...,51.0,0


In [13]:
df_annotation['annotation_label'] = df_annotation.apply(lambda row: 1 if row['Title 1'] == row['Title 2'] else 0, axis=1)

In [14]:
df_annotation['annotation_label'].sum()/len(df_annotation)

0.0

In [16]:
df_annotation[df_annotation['annotation_label']==1]


,Title 1,Title 2,Similarity Score,annotation_label


# match based on length of title

In [15]:

import numpy as np
with open('/content/drive/MyDrive/finalMetaFile/counterVect_L1_2kPair_annotation_similarLengthTitleUpto3Words.csv', 'w', newline='') as file:
  writer = csv.writer(file)
  writer.writerow(["Title 1", "Title 2", "Similarity Score"])
# Iterate over title pairs
  for i, v1 in enumerate(vects):
    if sum(v1[:-1]) > min_length: # Only above a minimum number of printable characters
        for j, v2 in enumerate(vects):
            if sum(v2[:-1]) > min_length:
                if i!=j:
                    if abs(len(v1[:-1]) - len(v2[:-1])) <= 3:
                        l1 = np.linalg.norm((v1[:-1] - v2[:-1]), ord=1)
                        if l1 < threshold:
                            t1 = df_make_pair.iloc[i].title
                            t2 = df_make_pair.iloc[j].title
                            writer.writerow([t1, t2, l1])
                            print(f"{t1} _&_ {t2} (L1: {l1:.0f})")


neither a zinc supplement nor phytate-reduced maize nor their combination enhance growth of 6- to 12-month-old guatemalan infants _&_ the effects of social norms on motivation crowding: experimental evidence from the tibetan plateau (L1: 45)
neither a zinc supplement nor phytate-reduced maize nor their combination enhance growth of 6- to 12-month-old guatemalan infants _&_ evaluating the environmental impact of payments for ecosystem services in coatepec (mexico) using remote sensing and on-site interviews (L1: 54)
neither a zinc supplement nor phytate-reduced maize nor their combination enhance growth of 6- to 12-month-old guatemalan infants _&_ saving lives with pre-arranged disaster aid: evidence from mexico (L1: 80)
neither a zinc supplement nor phytate-reduced maize nor their combination enhance growth of 6- to 12-month-old guatemalan infants _&_ conditional cash transfers and child labor (L1: 89)
neither a zinc supplement nor phytate-reduced maize nor their combination enhance gr

In [17]:
df_annotation_simialrLength = pd.read_csv(os.path.join(directory, 'counterVect_L1_2kPair_annotation_similarLengthTitleUpto3Words.csv'))
df_annotation_simialrLength['annotation_label']=0
df_annotation_simialrLength.head()

,Title 1,Title 2,Similarity Score,annotation_label
0,neither a zinc supplement nor phytate-reduced ...,the effects of social norms on motivation crow...,45.0,0
1,neither a zinc supplement nor phytate-reduced ...,evaluating the environmental impact of payment...,54.0,0
2,neither a zinc supplement nor phytate-reduced ...,saving lives with pre-arranged disaster aid: e...,80.0,0
3,neither a zinc supplement nor phytate-reduced ...,conditional cash transfers and child labor,89.0,0
4,neither a zinc supplement nor phytate-reduced ...,do households gain from community-based natura...,51.0,0


In [18]:
df_annotation_simialrLength['annotation_label'] = df_annotation_simialrLength.apply(lambda row: 1 if row['Title 1'] == row['Title 2'] else 0, axis=1)
df_annotation_simialrLength['annotation_label'].sum()/len(df_annotation_simialrLength)

0.0

In [19]:
df_annotation_simialrLength[df_annotation_simialrLength['annotation_label']==1]


,Title 1,Title 2,Similarity Score,annotation_label


In [20]:
len(df_annotation_simialrLength)

2256